<a href="https://colab.research.google.com/github/GursimranKaur12/Breast-Cancer-Prediction-Using-Neural-Networks-in-PyTorch/blob/main/Breas_Cancer_Prediction_Using_Neural_Network_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [15]:
data = load_breast_cancer()
X = data.data
Y = data.target

In [16]:
print(X)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [17]:
print(Y[:5])

[0 0 0 0 0]


In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size =0.2, random_state=42)

In [19]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(569, 30)
(455, 30)
(114, 30)


In [20]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
type(X_train)

numpy.ndarray

In [22]:
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
Y_train = torch.tensor(Y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
Y_test = torch.tensor(Y_test, dtype=torch.float32).to(device)

In [34]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid() # Changed nn.sigmoid() to nn.Sigmoid()

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out

In [35]:
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100

In [36]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)


In [37]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [39]:
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(X_train)
  loss = criterion(outputs, Y_train.view(-1,1))
  loss.backward()
  optimizer.step()

  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == Y_train.view(-1,1)).float().sum()
    accuarcy = correct/Y_train.size(0)
  if (epoch+1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy: {accuarcy.item()* 100:.2f}%")


Epoch [10/100], Loss : 0.4277, Accuracy: 91.21%
Epoch [20/100], Loss : 0.3366, Accuracy: 91.65%
Epoch [30/100], Loss : 0.2678, Accuracy: 92.31%
Epoch [40/100], Loss : 0.2183, Accuracy: 94.95%
Epoch [50/100], Loss : 0.1835, Accuracy: 95.60%
Epoch [60/100], Loss : 0.1584, Accuracy: 96.04%
Epoch [70/100], Loss : 0.1397, Accuracy: 96.26%
Epoch [80/100], Loss : 0.1253, Accuracy: 97.36%
Epoch [90/100], Loss : 0.1139, Accuracy: 97.36%
Epoch [100/100], Loss : 0.1045, Accuracy: 97.80%


In [41]:
model.eval()
with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()
  correct = (predicted == Y_train.view(-1, 1)).float().sum()
  accuracy = correct/Y_train.size(0)
  print(f"Training Accuarcy: {accuracy.item() * 100:.2f}%")

Training Accuarcy: 97.80%


In [42]:
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  correct = (predicted == Y_test.view(-1, 1)).float().sum()
  accuracy = correct/Y_test.size(0)
  print(f"Accuarcy on test data: {accuracy.item() * 100:.2f}%")

Accuarcy on test data: 98.25%
